## Gait Video Study 
### 1D Convolutional neural network (CNN) on task generalization framework 1: train on walking (W) and test on walking while talking (WT) to classify HOA/MS/PD strides and subjects 
#### Remember to add the original count of frames in a single stride (before down sampling via smoothing) for each stride as an additional artificial feature to add information about speed of the subject to the model

1. Save the optimal hyperparameters, confusion matrices and ROC curves for each algorithm.
2. Make sure to not use x, y, z, confidence = 0, 0, 0, 0 as points for the model since they are simply missing values and not data points, so make sure to treat them before inputting to model 
3. Make sure to normalize (z-score) the features before we feed them to the model.
4. Make sure to set a random seed wherever required for reproducible results.


In [1]:
from importlib import reload
import imports 
reload(imports)
from imports import *

### Utility functions 

In [2]:
def set_random_seed(seed_value, use_cuda):
    '''
    To set the random seed for reproducibility of results 
    Arguments: seed value and use cuda (True if cuda is available)
    '''
    random.seed(seed_value)
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    if use_cuda: 
        torch.cuda.manual_seed_all(seed_value) # gpu vars

In [3]:
def list_subjects_common_across_train_test(trial_train, trial_test):
    '''
    Since we need to implement pure task generalization framework, we must have same subjects across both training and testing trails 
    Hence, if there are some subjects that are present in the training set but not in the test set or vice versa, we eliminate those 
    subjects to have only common subjects across training and test sets. 
    Arguments: data subset for training and testing trial
    Returns: PIDs to retain in the training and test subsets with common subjects 
    '''
    
    print ('Original number of subjects in training and test sets:', len(trial_train['PID'].unique()), len(trial_test['PID'].unique()))

    #Try to use same subjects in trials W and WT for testing on same subjects we train on
    print ('Subjects in test set, which are not in training set')
    pids_missing_training = [] #PIDs missing in training set (trial W) but are present in the test set (trial WT)
    for x in trial_test['PID'].unique():
        if x not in trial_train['PID'].unique():
            pids_missing_training.append(x)
    print (pids_missing_training)
    #List of PIDs to retain in the training set 
    pids_retain_training = [i for i in trial_test['PID'].unique() if i not in pids_missing_training]
    
    print ('Subjects in training set, which are not in test set')
    pids_missing_test = [] #PIDs missing in test set (trial WT) but are present in the training set (trial W)
    for x in trial_train['PID'].unique():
        if x not in trial_test['PID'].unique():
            pids_missing_test.append(x)
    print (pids_missing_test)
    #List of PIDs to retain in the testing set 
    pids_retain_test = [i for i in trial_train['PID'].unique() if i not in pids_missing_test]
    
    print ('Number of subjects in training and test sets after reduction:', len(pids_retain_training), \
           len(pids_retain_test))
    #Returning the PIDs to retain in the training and test set
    return  pids_retain_training, pids_retain_test

In [4]:
#Pytorch dataset definition
class GaitDataset(Dataset):
    #We need to add the frame count as an extra feature along with 36 features for each stride 
    def __init__(self, data_path, labels_csv, pids_retain, framework = 'W', transforms = None, train_data_mean = None, train_data_std = None):   
        '''
        Arguments: 
        data_path: data path for downsampled strides 
        labels_csv: csv file with labels 
        pids_retain: PIDs to return data for 
        framework: Task to return data for 
        transforms: For ToTensor transformation of dataframes
        train_data_mean: Mean for the training data (computed beforehand) to z-score normalize the training and testing samples 
        train_data_std: Standard deviation for the training data (computed beforehand) to z-score normalize the training and testing samples
        
        Returns:
        X: 20 rows for 20 downsampled frames per stride and 37 columns for 37 features for each sample. The features are z-score 
        normalized and converted to tensor.
        y: PID and label for each sample. These values are converted to tensor.
        '''
        #Assigning the data folder for the downsampled strides 
        self.data_path = data_path
        #Reading the labels file
        self.all_labels = pd.read_csv(labels_csv, index_col = 0)
        #Retaining only the labels dataframe for framework and PIDs of interest and resetting the index
        self.reduced_labels = self.all_labels[self.all_labels.scenario == framework][self.all_labels.PID.isin(pids_retain)].reset_index()
        #Setting the labels with index as the key and PID along with to use when computing subject wise evaluation metrics
        self.labels = self.reduced_labels[['PID', 'label', 'key']].set_index('key')
        self.len = len(self.labels) #Length of the data to use
        self.transforms = transforms
        self.train_data_mean = train_data_mean
        self.train_data_std = train_data_std
    
    def __len__(self):
        #Returns the length of the data 
        return self.len
    
    def __getitem__(self, index):
#         print ('mean = ', self. train_data_mean)
        #Generates one sample of data
        #Select key to sample
        key = self.reduced_labels['key'].iloc[index]

        # Load data and get label
        X = pd.read_csv(data_path+key+'.csv', index_col = 0)
        #Creating a new frame count column represting the total original count of frames in a stride 
        #denoting the speed of the stride
        X['frame_count'] = self.reduced_labels[self.reduced_labels['key']==key]['frame_count'].values[0]
        y = self.labels.loc[key] #PID and label extracted for the key at the index 
        #X- 20 rows for 20 downsampled frames per stride and 37 columns for 37 features for each sample
        #y - PID and label for each sample
        if self.transforms is not None: #Used for loading the data all at once to compute feature-wise mean and std for z-score 
            X = self.transforms(X.values).squeeze() #20*37 for 20 downsampled frames and 37 features in each frame 
        else: #Used to load z-score normalized data in batches 
            X = (X-self.train_data_mean)/self.train_data_std #z-score normalization for all 37 features 
            X = torch.Tensor(X.values) #converting the dataframe to tensor 
        y = torch.Tensor(y) #shape = 2 for PID and label 
#         print (X.shape, y.shape)
        return X, y
    

In [5]:
def get_data_loaders(data_path, labels_file, pids_retain_train, pids_retain_test, parameter_dict, train_framework, test_framework):
    '''
    To define the training and testing data loader to load X, y for training and testing sets in batches 
    Arguments:
        data_path: path for downsampled frame csvs
        labels_file: path for labels file with key and labels 
        pids_retain_train: Based on the framework, PIDs to extract training data for 
        pids_retain_test: Based on the framework, PIDs to extract testing data for 
        parameter_dict: dictionary with parameters defined 
        train_framework: task to train on 
        test_framework: task to test on 
    Returns: 
        training_data_loader, testing_data_loader to load X, y for training and testing sets in batches 
    '''
    #Loading the full training data in one go to compute the training data's mean and standard deviation for normalization 
    #We set the batch_size = len(training_data) for the same 
    training_data = GaitDataset(data_path, labels_file, pids_retain_train, framework = train_framework, \
                                transforms=transforms.Compose([transforms.ToTensor()]))   
    training_data_loader = DataLoader(training_data, batch_size = len(training_data), shuffle = parameter_dict['shuffle'], \
                                      num_workers = parameter_dict['num_workers'])
    #Since we loaded all the training data in a single batch, we can read all data and target in one go
    data, target = next(iter(training_data_loader))
    #Computing the training data mean and standard deviation
    training_data_mean = data.reshape(data.shape[0]*data.shape[1], data.shape[2]).mean(axis = 0)
    training_data_std = data.reshape(data.shape[0]*data.shape[1], data.shape[2]).std(axis = 0)
    print ('Training data mean: ', training_data_mean, training_data_mean.shape)
    print ('Training data standard deviation: ', training_data_std, training_data_std.shape)

    #With training data mean and standard deviation computed, we can load the z-score normalized training and testing data in batches 
    training_data = GaitDataset(data_path, labels_file, pids_retain_train, framework = train_framework, \
                                train_data_mean=training_data_mean, train_data_std=training_data_std)   
    testing_data = GaitDataset(data_path, labels_file, pids_retain_test, framework = test_framework, \
                              train_data_mean=training_data_mean, train_data_std=training_data_std) 

    training_data_loader = DataLoader(training_data, batch_size = parameter_dict['batch_size'], shuffle = parameter_dict['shuffle'], \
                                      num_workers = parameter_dict['num_workers'])
    testing_data_loader = DataLoader(testing_data, batch_size = parameter_dict['batch_size'], shuffle = parameter_dict['shuffle'], \
                                     num_workers = parameter_dict['num_workers'])
    
    #To make sure the z-score normalization worked correctly 
    training_data_loader_check = DataLoader(training_data, batch_size = len(training_data), shuffle = parameter_dict['shuffle'], \
                                      num_workers = parameter_dict['num_workers'])
    data_check, target_check= next(iter(training_data_loader_check))
    #The normalized means for each of the 37 features must be ~0
    print ('Normalized training data\'s mean:', data_check.reshape(data_check.shape[0]*data_check.shape[1], data_check.shape[2]).mean(axis = 0))
    #The normalized standard deviations for each of the 37 feature must be ~1
    print ('Normalized training data\'s standard deviation:', data_check.reshape(data_check.shape[0]*data_check.shape[1], data_check.shape[2]).std(axis = 0))
    
    return training_data_loader, testing_data_loader

In [6]:
def evaluate(model, test_features, trueY, framework, model_name):
    '''
    Function to evaluate ML models and plot it's confusion matrix
    Input: model, test set, true test set labels, framework name, model name
    Computes the stride and subject wise test set evaluation metrics 
    Returns: Prediction probabilities for HOA/MS/PD and stride and subject wise evaluation metrics 
    (Accuracy, Precision, Recall, F1 and AUC)
    '''
    test_labels = trueY['label'] #Dropping the PID
#     print ('Test labels', test_labels)
    predictions = model.predict(test_features)
#     print ('Predictions', predictions)
    
    #Stride wise metrics 
    acc = accuracy_score(test_labels, predictions)
    #For multiclass predictions, we need to use marco/micro average
    p = precision_score(test_labels, predictions, average='macro')  
    r = recall_score(test_labels, predictions, average = 'macro')
    f1 = f1_score(test_labels, predictions, average= 'macro')
    
    try:
        prediction_prob = model.predict_proba(test_features) #Score of the class with greater label
#         print ('Prediction Probability', model.predict_proba(test_features))
        
    except:
        prediction_prob = model.best_estimator_._predict_proba_lr(test_features) #For linear SVM
#         print ('Prediction Probability', model.best_estimator_._predict_proba_lr(test_features))
    
    #For computing the AUC, we would need prediction probabilities for all the 3 classes 
    auc = roc_auc_score(test_labels, prediction_prob, multi_class = 'ovo', average= 'macro')
    print('Stride-based model performance: ', acc, p, r, f1, auc)
    
    #For computing person wise metrics 
    temp = copy.deepcopy(trueY) #True label for the stride 
    temp['pred'] = predictions #Predicted label for the stride 
    #Saving the stride wise true and predicted labels for calculating the stride wise confusion matrix for each model
    temp.to_csv(results_path+ framework + '\\stride_wise_predictions_' + str(model_name) + '_' + framework + '.csv')
    
    x = temp.groupby('PID')['pred'].value_counts().unstack()
    #Input for subject wise AUC is probabilities at columns [0, 1, 2]
    proportion_strides_correct = x.divide(x.sum(axis = 1), axis = 0).fillna(0) 
    proportion_strides_correct['True Label'] = trueY.groupby('PID').first()
    #Input for precision, recall and F1 score
    proportion_strides_correct['Predicted Label'] = proportion_strides_correct[[0, 1, 2]].idxmax(axis = 1) 
    #Saving the person wise true and predicted labels for calculating the subject wise confusion matrix for each model
    proportion_strides_correct.to_csv(results_path+ framework + '\\person_wise_predictions_' + \
                                      str(model_name) + '_' + framework + '.csv')
    try:
        print (model.best_estimator_)
    except:
        pass
    #Person wise metrics 
    person_acc = accuracy_score(proportion_strides_correct['True Label'], proportion_strides_correct['Predicted Label'])
    person_p = precision_score(proportion_strides_correct['True Label'], proportion_strides_correct['Predicted Label'], \
                               average = 'macro')
    person_r = recall_score(proportion_strides_correct['True Label'], proportion_strides_correct['Predicted Label'], \
                            average = 'macro')
    person_f1 = f1_score(proportion_strides_correct['True Label'], proportion_strides_correct['Predicted Label'], \
                         average = 'macro')
    person_auc = roc_auc_score(proportion_strides_correct['True Label'], proportion_strides_correct[[0, 1, 2]], \
                               multi_class = 'ovo', average= 'macro')
    print('Person-based model performance: ', person_acc, person_p, person_r, person_f1, person_auc)
      
    #Plotting and saving the subject wise confusion matrix 
    plt.figure()
    confusion_matrix = pd.crosstab(proportion_strides_correct['True Label'], proportion_strides_correct['Predicted Label'], \
                                   rownames=['Actual'], colnames=['Predicted'], margins = True)
    sns.heatmap(confusion_matrix, annot=True, cmap="YlGnBu")
    plt.savefig(results_path + framework+'\\CFmatrix_task_generalize_' + framework + '_'+ ml_model+ '.png', dpi = 350)
    plt.show()
    return proportion_strides_correct[[0, 1, 2]], [acc, p, r, f1, auc, person_acc, person_p, person_r, person_f1, person_auc] 

In [7]:
#Test set ROC curves for cohort prediction 
def plot_ROC(ml_models, testY, predicted_probs_person, framework):
    '''
    Function to plot the ROC curve for models given in ml_models list 
    Input: ml_models (name of models to plot the ROC for),  test_Y (true test set labels with PID), 
        predicted_probs_person (predicted test set probabilities for all 3 classes - HOA/MS/PD), framework (WtoWT / VBWtoVBWT)
    Plots and saves the ROC curve with individual class-wise plots and micro/macro average plots 
    '''
    n_classes = 3 #HOA/MS/PD
    cohort = ['HOA', 'MS', 'PD']
    ml_model_names = {'random_forest': 'RF', 'adaboost': 'Adaboost', 'kernel_svm': 'RBF SVM', 'gbm': 'GBM', \
                      'xgboost': 'Xgboost', 'knn': 'KNN', 'decision_tree': 'DT',  'linear_svm': 'LSVM', 
                 'logistic_regression': 'LR', 'mlp': 'MLP'}
    #PID-wise true labels 
    person_true_labels = testY.groupby('PID').first()
    #Binarizing/getting dummies for the true labels i.e. class 1 is represented as 0, 1, 0
    person_true_labels_binarize = pd.get_dummies(person_true_labels.values.reshape(1, -1)[0])  

    sns.despine(offset=0)
    linestyles = ['-', '-', '-', '-.', '--', '-', '--', '-', '--']
    colors = ['b', 'magenta', 'cyan', 'g',  'red', 'violet', 'lime', 'grey', 'pink']
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for idx, ml_model in enumerate(ml_models): #Plotting the ROCs for all models in ml_models list
        fig, axes = plt.subplots(1, 1, sharex=True, sharey = True, figsize=(6, 4.5))
        axes.plot([0, 1], [0, 1], linestyle='--', label='Majority (AUC = 0.5)', linewidth = 3, color = 'k')
        # person-based prediction probabilities for class 0: HOA, 1: MS, 2: PD
        model_probs = predicted_probs_person[[ml_model+'_HOA', ml_model+'_MS', ml_model+'_PD']]

        for i in range(n_classes): #For 3 classes 0, 1, 2
            fpr[i], tpr[i], _ = roc_curve(person_true_labels_binarize.iloc[:, i], model_probs.iloc[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i]) #Computing the AUC score for each class
            #Plotting the ROCs for the three classes separately
            axes.plot(fpr[i], tpr[i], label = cohort[i] +' ROC (AUC = '+ str(round(roc_auc[i], 3))
                +')', linewidth = 3, alpha = 0.8, linestyle = linestyles[i], color = colors[i]) 

        # Compute micro-average ROC curve and ROC area (AUC)
        fpr["micro"], tpr["micro"], _ = roc_curve(person_true_labels_binarize.values.ravel(), model_probs.values.ravel())
        #Micro average AUC of ROC value
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"]) 
        #Plotting the micro average ROC 
        axes.plot(fpr["micro"], tpr["micro"], label= 'micro average ROC (AUC = '+ str(round(roc_auc["micro"], 3))
                +')', linewidth = 3, alpha = 0.8, linestyle = linestyles[3], color = colors[3])

        #Compute the macro-average ROC curve and AUC value
        all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)])) # First aggregate all false positive rates
        mean_tpr = np.zeros_like(all_fpr) # Then interpolate all ROC curves at this points
        for i in range(n_classes):
            mean_tpr += interp(all_fpr, fpr[i], tpr[i])
        mean_tpr /= n_classes  # Finally average it and compute AUC
        fpr["macro"] = all_fpr
        tpr["macro"] = mean_tpr
        #Macro average AUC of ROC value 
        roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
        #Plotting the macro average AUC
        axes.plot(fpr["macro"], tpr["macro"], label= 'macro average ROC (AUC = '+ str(round(roc_auc["macro"], 3))
            +')', linewidth = 3, alpha = 0.8, linestyle = linestyles[4], color = colors[4])

        axes.set_ylabel('True Positive Rate')
        axes.set_title('Task generalization '+framework + ' '+ ml_model_names[ml_model])
        plt.legend()
        # axes[1].legend(loc='upper center', bbox_to_anchor=(1.27, 1), ncol=1)

        axes.set_xlabel('False Positive Rate')
        plt.tight_layout()
        plt.savefig(results_path + framework+'\\ROC_task_generalize_' + framework + '_'+ ml_model+ '.png', dpi = 350)
        plt.show()

### main()

In [8]:
parameter_dict = {'path': 'C:\\Users\\Rachneet Kaur\\Box\\Gait Video Project\\GaitVideoData\\video\\',
                  'data_path': 'downsampled_strides\\', 
                  'labels_file': 'labels.csv',
                  'results_path': 'C:\\Users\\Rachneet Kaur\\Box\\Gait Video Project\\DLresults\\CNN1D\\',
                  'batch_size': 100,
                  'shuffle':True,
                  'num_workers':0 
                  #num_workers zero means the loader loads the data inside the main process 
                  #i.e. training process will work sequentially inside the main process.
                 }                
use_cuda = torch.cuda.is_available() #use_cuda is True if cuda is available 
set_random_seed(0, use_cuda) #Setting a fixed random seed for reproducibility 

#### Task generalization framework 1: train on walking (W) and test on walking while talking (WT) to classify HOA/MS/PD strides and subjects 

In [ ]:
#Task generalization W-> WT framework 
data_path = parameter_dict['path'] + parameter_dict['data_path']
labels_file = parameter_dict['path'] + parameter_dict['labels_file']

labels = pd.read_csv(labels_file, index_col = 0)
#Trial W for training 
trialW = labels[labels['scenario']=='W']
#Trial WT for testing 
trialWT = labels[labels['scenario']=='WT']
#Returning the PIDs of common subjects in training and testing set
pids_retain_trialW, pids_retain_trialWT = list_subjects_common_across_train_test(trialW, trialWT)
#Note that both pids_retain_trialW, pids_retain_trialWT will be the same since we are only retaining common subjects 
#in training and testing trials for a "pure" task generalization framework

#Showing the statistics and imbalance ratio of training and testing data 
trialW_reduced = trialW[trialW.PID.isin(pids_retain_trialW)]
print ('Strides in training set: ', trialW_reduced.shape[0])
print ('HOA, MS and PD strides in training set:\n', trialW_reduced['cohort'].value_counts())

trialWT_reduced = trialWT[trialWT.PID.isin(pids_retain_trialWT)]
print ('Strides in testing set: ', trialWT_reduced.shape[0])
print ('HOA, MS and PD strides in testing set:\n', trialWT_reduced['cohort'].value_counts())
print ('Imbalance ratio (controls:MS:PD)= 1:X:Y\n', trialWT_reduced['cohort'].value_counts()/trialWT_reduced['cohort'].value_counts()['HOA'])

framework = 'WtoWT' #Defining the task generalization framework of interest

Original number of subjects in training and test sets: 32 26
Subjects in test set, which are not in training set
[403]
Subjects in training set, which are not in test set
[312, 102, 112, 113, 115, 123, 124]
Number of subjects in training and test sets after reduction: 25 25
Strides in training set:  1128
HOA, MS and PD strides in training set:
 PD     453
MS     341
HOA    334
Name: cohort, dtype: int64
Strides in testing set:  1142
HOA, MS and PD strides in testing set:
 PD     459
HOA    351
MS     332
Name: cohort, dtype: int64
Imbalance ratio (controls:MS:PD)= 1:X:Y
 PD     1.307692
HOA    1.000000
MS     0.945869
Name: cohort, dtype: float64


In [ ]:
#To define the training and testing data loader to load X, y for training and testing sets in batches 
training_data_loader, testing_data_loader = get_data_loaders(data_path, labels_file, pids_retain_trialW, \
                                                             pids_retain_trialWT, parameter_dict, 'W', 'WT')
print ('No. of batches in training loader', len(training_data_loader))
print ('No. of batches in testing loader', len(testing_data_loader))

In [ ]:
class CNN1D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(CNN1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=7, out_channels=20, kernel_size=5, stride=2)
        self.conv2 = nn.Conv1d(in_channels=20, out_channels=10, kernel_size=1)
        self.bn1 = nn.BatchNorm2d(128)
        self.dropout = nn.Dropout(0.5)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        log_probs = F.log_softmax(x, dim=1)
        return log_probs

In [ ]:
for batch_idx, (data, target) in enumerate(training_data_loader):
    print (data)
    print (target)
    break

In [ ]:
X = numpy.random.uniform(-10, 10, 70).reshape(1, 7, -1)
# Y = np.random.randint(0, 9, 10).reshape(1, 1, -1)

# Hyperparameters
num_epochs = 5
num_classes = 3
batch_size = 100
learning_rate = 0.001

# loss 
criterion = nn.CrossEntropyLoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#Error messages are much better if on CPU
#device = torch.device("cpu")
print(device)

model = LSTM(input_size, hidden_size, num_layers, num_classes).to(device)

criterion = nn.BCELoss()
#criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for batch_idx, (data, preds) in enumerate(trainDataloader):
        #Get data to cuda if possible
        data = data.to(device=device).squeeze(1)
        #print("data nan? ", (torch.isnan(data)).any())
        #print(data)
        #print("original data shape: ",data.shape)
        label = preds.to(device=device)
        #print(label)
        #print("original label shape: ",label.shape)

        # forward
        #print(targets)
        pred = model(data.float())
        pred = torch.squeeze(pred)
        #print("post squeeze output shape: ", pred.shape)
        loss = criterion(pred, label.float())
        

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    #if loader.dataset.train:
    #    print("Checking accuracy on training data")
    #else:
    #    print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0

    # Set model to eval
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x.float())
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with \
              accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )
    # Set model back to train
    model.train()

model = CNN1D().double()
print(model(torch.tensor(X)).shape)
torch.save(model.state_dict(), results_path+framework+'\\'+ model_save_name+ '.pt')
# model = cnn_model.ConvNet(32).double()
# model.cuda()
# model.load_state_dict(torch.load(results_path+framework+'\\'+ model_save_name+ '.pt'))

#Maybe use skorch for hyperparameter grid search 

In [ ]:
ml_model = 'CNN1D'
metrics = pd.DataFrame(columns = ml_model) #Dataframe to store accuracies for each ML model for raw data 

#For storing predicted probabilities for person (for all classes HOA/MS/PD) to show ROC curves 
predicted_probs_person = pd.DataFrame(columns = [ml_model + cohort for cohort in ['_HOA', '_MS', '_PD'] ]) 

print (ml_model)
predict_probs_person, stride_person_metrics = models(trainX_norm, trainY, testX_norm, testY, ml_model, framework)  
metrics[ml_model] = stride_person_metrics
predicted_probs_person[ml_model+'_HOA'] = predict_probs_person[0]
predicted_probs_person[ml_model+'_MS'] = predict_probs_person[1]
predicted_probs_person[ml_model+'_PD'] = predict_probs_person[2]
print ('********************************')

metrics.index = ['stride_accuracy', 'stride_precision', 'stride_recall', 'stride_F1', 'stride_AUC', 'person_accuracy', 
                     'person_precision', 'person_recall', 'person_F1', 'person_AUC']  
metrics.to_csv(results_path+'task_generalize_'+framework+'_result_metrics.csv')
predicted_probs_person.to_csv(results_path+'task_generalize_'+framework+'_prediction_probs.csv')

In [ ]:
plot_ROC(ml_models, testY, predicted_probs_person, framework)